# GRAPH PLOT

In [ ]:
main_path = ""

In [12]:
platform2parameters = {
    "cross": {
        "url": {
            "MIN_URLS_PER_USER": 10, # distinct urls
            "MIN_DF": 5,
            "MAX_DF": 0.9,
            "MIN_SIMILARITY_PERCENTILE": 99,#x
            "MIN_CENTRALITY_PERCENTILE": 95,#y
            "MIN_COURLS": 0,
        }
    },
    "telegram": {
        "url": {
            "MIN_URLS_PER_USER": 10, # distinct urls
            "MIN_DF": 5,
            "MAX_DF": 0.9,
            "MIN_SIMILARITY_PERCENTILE": 99,#x
            "MIN_CENTRALITY_PERCENTILE": 99,#y
            "MIN_COURLS": 0,
        }
    },
    "twitter": {
        "url": {
            "MIN_URLS_PER_USER": 10, # distinct urls
            "MIN_DF": 5,
            "MAX_DF": 0.9,
            "MIN_SIMILARITY_PERCENTILE": 85, #x
            "MIN_CENTRALITY_PERCENTILE": 99, #y
            "MIN_COURLS": 0,
        }
    },
    "facebook": {
        "url": {
            "MIN_URLS_PER_USER": 5, # distinct urls
            "MIN_DF": 3,
            "MAX_DF": 0.9,
            "MIN_SIMILARITY_PERCENTILE": 50,#x
            "MIN_CENTRALITY_PERCENTILE": 45,#y
            "MIN_COURLS": 0,
        }
    }
}

In [13]:

# https://github.com/chan0park/VoynaSlov/blob/master/media-accounts/VK/state-affiliated.txt
russia_media = ['rt_russian',
'sputnikints',
'tassagency',
'mil',
'ria',
'ruptly',
'm24',
'ukraina_ru_official',
'1prime',
'inosmi',
'life',
'sputnik_radio',
'tv5',
'vesti',
'russiatv',
'rbc',
'gazeta',
'rgru',]

In [4]:
from pyvis.network import Network
import networkx as nx

# Define colors for each platform suffix
platform_colors = {
    "facebook": "blue",
    "twitter": "lightblue",
    "telegram": "green",
}
platform2coordinated_users = {
    "facebook": [],
    "twitter":  [],
    "telegram": [],
    "cross":    []
}

TYPE_OF_NETWORK = "url"

# Initialize an empty graph to store the unified graph
unified_graph = nx.Graph()

# Iterate over each platform, create the filename, load the graph, and combine
for platform, params in platform2parameters.items():
    # Hash the parameters into a filename
    hashed_params = "-".join([f"{k}_{v}" for k, v in params[TYPE_OF_NETWORK].items()])
    output_name = f"{platform}-coordinated-params-{hashed_params}"
    filename = f"./cross-platform/grid-search/{output_name}/{output_name}-networkx-graph.gexf"
    
    # Load the graph from the GEXF file
    try:
        graph = nx.read_gexf(filename)
        print(f"\n✅ Loaded graph for {platform} from {filename}: \n{graph.number_of_nodes():,} nodes")
        
        # Rename nodes based on platform and assign colors
        new_names = {}
        for node in graph.nodes():
            if platform == "facebook":
                new_names[node] = f"{node}_facebook_page"
            elif platform == "twitter":
                new_names[node] = f"{node}_twitter_user"
            elif platform == "telegram":
                new_names[node] = f"{node}_telegram_channel"
            elif platform == "cross":
                new_names[node] = node  # Keep it as is for cross-platform
            platform2coordinated_users[platform].append(node)
        
        # Apply the renaming
        graph = nx.relabel_nodes(graph, new_names)

        # Remove those nodes from the graph
        nodes_to_remove = [node for node in graph.nodes()]
        graph.remove_nodes_from(nodes_to_remove)
        
        # Assign color based on the node's suffix
        for node in graph.nodes():
            # Extract the suffix (e.g., "facebook_page", "twitter_user") after the last '_'
            suffix = node.split('_')[-2]
            # Assign the color based on the suffix, default to gray if not found
            color = platform_colors.get(suffix, "gray")
            graph.nodes[node]["color"] = color  # Assign color to each node
            graph.nodes[node]["platform_label"] = suffix

        # Combine the loaded graph with the unified graph
        unified_graph.add_edges_from(graph.edges(data=False))

        # Add nodes from the graph to the unified_graph with their attributes
        unified_graph.add_nodes_from(graph.nodes(data=True))
        
    except FileNotFoundError:
        print(f"\n❌ File {filename} not found. Skipping {platform}.")

# Visualize the unified graph with pyvis
net = Network(width="100%", height="750px")

# Load the unified graph into pyvis Network object
net.from_nx(unified_graph)

# Save the graph to an HTML file
net.save_graph("cross-platform-coordinated.html")

print(f"Unified graph saved as 'cross-platform-coordinated.html'. Open it in a web browser to view.")

# Print the final number of nodes and edges
print(f"Number of nodes: {unified_graph.number_of_nodes():,}")
print(f"Number of edges: {unified_graph.number_of_edges():,}")



✅ Loaded graph for cross from ./cross-platform/grid-search/cross-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MAX_DF_0.9-MIN_SIMILARITY_PERCENTILE_99-MIN_CENTRALITY_PERCENTILE_95-MIN_COURLS_0/cross-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MAX_DF_0.9-MIN_SIMILARITY_PERCENTILE_99-MIN_CENTRALITY_PERCENTILE_95-MIN_COURLS_0-networkx-graph.gexf: 
184 nodes

✅ Loaded graph for telegram from ./cross-platform/grid-search/telegram-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MAX_DF_0.9-MIN_SIMILARITY_PERCENTILE_99-MIN_CENTRALITY_PERCENTILE_99-MIN_COURLS_0/telegram-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MAX_DF_0.9-MIN_SIMILARITY_PERCENTILE_99-MIN_CENTRALITY_PERCENTILE_99-MIN_COURLS_0-networkx-graph.gexf: 
33 nodes

✅ Loaded graph for twitter from ./cross-platform/grid-search/twitter-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MAX_DF_0.9-MIN_SIMILARITY_PERCENTILE_85-MIN_CENTRALITY_PERCENTILE_99-MIN_COURLS_0/twitter-coordinated-params-MIN_URLS_PER_USER_10-MIN_DF_5-MA

In [5]:
from pathlib import Path
import pickle

main_path = Path(f"{main_path}/")
with open(main_path / "co-url-coordinated", "wb") as f:
    pickle.dump(platform2coordinated_users, f)

In [6]:
from pyvis.network import Network
import networkx as nx

# Create the pyvis Network object
net = Network(width="100%", height="750px")#, select_menu=False, filter_menu=True,)

# Check if the unified graph has nodes and edges
print(f"Number of nodes: {unified_graph.number_of_nodes()}")
print(f"Number of edges: {unified_graph.number_of_edges()}")

# Load the networkx graph into the pyvis Network object
net.from_nx(unified_graph)
net.toggle_physics(False)
net.show_buttons(filter_=['physics'])
net.save_graph("cross-platform-coordinated.html")
output_combined_filename = "./cross-platform-coordinated-gephi.gexf"
nx.write_gexf(unified_graph, output_combined_filename)

Number of nodes: 231
Number of edges: 3706


-------

# DOMAINS TABLES

In [2]:
from collections import Counter

def count_unique_terms(df, column_name):
    """
    Given a DataFrame and a column containing lists of strings, generate a DataFrame
    that counts the occurrence of each unique string term.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing lists of strings.

    Returns:
        pd.DataFrame: A DataFrame with columns 'value' and 'count' representing
                      each unique string term and its count.
    """
    # Flatten the list of all strings in the specified column
    all_strings = [item for sublist in df[column_name] for item in sublist]
    
    # Count the occurrences of each unique term
    term_counts = Counter(all_strings)
    
    # Create a DataFrame from the counts
    result_df = pd.DataFrame(term_counts.items(), columns=['value', 'count'])
    
    return result_df

def merge_values_by_case_insensitivity(df):
    """
    Given a DataFrame with columns [value, count], merge entries where 
    the 'value' differs only by capitalization.

    Args:
        df (pd.DataFrame): The input DataFrame with columns 'value' and 'count'.

    Returns:
        pd.DataFrame: A DataFrame with merged entries, case-insensitively.
    """
    # Normalize the 'value' column to lowercase
    df['value'] = df['value'].str.lower()

    # Group by the normalized 'value' and sum the counts
    merged_df = df.groupby('value', as_index=False)['count'].sum()

    return merged_df

UNRELATED_NODES = {
    "telegram": {
'1159862323',
 '1370712701',
 '1379968896',
 '1385724061',
 '1467522018',
 '1543456510',
 '1568496729',
 '1678554747',
 '1791839714',
 '1819499842',
 '1835830789',
 '1838576670',
 '1880117389',
 '1890545978',
 '1890620459',
 '1942839236',
 '1188004204',
 '1336022000',
 '1343518359',
 '1355491057',
 '1358837396',
 '1410089638',
 '1437157101',
 '1461525797',
 '1483014277',
 '1502533790',
 '1543131980',
 '1560397340',
 '1562747318',
 '1606340767',
 '1620555236',
 '1663668688',
 '1681107467',
 '1683697086',
 '1691291673',
 '1714276779',
 '1755667684',
 '1780860644',
 '1788005462',
 '1795953302',
 '1797925558',
 '1801439162',
 '1802099608',
 '1802401461',
 '1806362406',
 '1814692877',
 '1815062639',
 '1821419847',
 '1831067702',
 '1833635084',
 '1837945457',
 '1842090694',
 '1847409035',
 '1849374936',
 '1849600895',
 '1854698309',
 '1855404132',
 '1855774684',
 '1857736893',
 '1858527115',
 '1861592664',
 '1865617712',
 '1867049107',
 '1870911488',
 '1871172553',
 '1876301109',
 '1879820206',
 '1881004262',
 '1883088850',
 '1884867194',
 '1886984973',
 '1893011233',
 '1897479788',
 '1897947114',
 '1911590958',
 '1916216693',
 '1937354777',
 '1938379379',
 '1946378324',
 '1952461446',
 '1962954847',
 '1966812415',
 '1975369864',
 '1980676178',
 '1984599717',
 '1995300780',
 '1996582667',
 '2000420557',
 '2000660679',
 '2001972260',
 '2002484709',
 '2003075134',
 '2003227802',
 '2004862535',
 '2005690092',
 '2006382925',
 '2007140139',
 '2010560852',
 '2013792173',
 '2015746082',
 '2017838249',
 '2018903157',
 '2020139762',
 '2024917189',
 '2026248623',
 '2030959379',
 '2031426561',
 '2032468473',
 '2032904128',
 '2036806678',
 '2038583939',
 '2043689675',
 '2045617403',
 '2059675395',
 '2061478565',
 '2066081358',
 '2067726505',
 '2068810045',
 '2071567209',
 '2072766397',
 '2077106817',
 '2080582377',
 '2081063200',
 '2081450649',
 '2085338139',
 '2090703801',
 '2091042268',
 '2096484482',
 '2098973756',
 '2104833233',
 '2112897929',
 '2118951661',
 '2120944773',
 '2124392949',
 '2126270142',
 '2126759719',
 '2134356909',
 '2141202046',
 '2142422700',
 '2143244497',
 '2146547416',
 '2146897797',
 '1510610641', 
 '1607397371', 
 '1710145932', 
 '1736249842', 
 '1761845091',
 '1426364128',
 '1493058778',
 '1506934839',
 '1507254194',
 '1512977397',
 '1525058352',
 '1533253538',
 '1554755636',
 '1565723991',
 '1576163324',
 '1587835005',
 '1594144610',
 '1598152350',
 '1159701555',
 '1235601643',
 '1241977683',
 '1323394034',
 '1515838925',
 '1526401436',
 '1528468158',
 '1538019465',
 '1613902974',
 '1614225503',
 '1634229073',
 '1668299023',
 '1697312650',
 '1701778429',
 '1714480274',
 '1779607856',
 '1540757260',
 '1578838051',
 '1598592631',
 '1609484736',
 '1780381150',
 '1987883378'
    },
    "twitter": {
        
    },
    "facebook": {
        
    }
}


from pathlib import Path
import polars as pl

def read_cross_dataframes(MIN_URLS_PER_USER=10, UNRELATED_NODES=UNRELATED_NODES, verbose=False):
    data_path = Path(f"{main_path}/telegram-clean/channel2urls_expanded_domains.parquet")
    df_telegram = pl.read_parquet(data_path).with_columns([
        pl.col('channel_id').cast(pl.Utf8)  # Cast user_id to string
    ])
    if verbose: display(df_telegram.head(2))
    if df_telegram['channel_id'].null_count():
        print("Nan users in Telegram")
    
    data_path = Path(f"{main_path}/twitter-clean/user_id2urls_expanded_domains.parquet")
    df_twitter = pl.read_parquet(data_path).with_columns([
        pl.col('user_id').cast(pl.Utf8)  # Cast user_id to string
    ])
    if verbose: display(df_twitter.head(2))
    if df_twitter['user_id'].null_count():
        print("Nan users in Twitter")

    
    
    data_path = Path(f"{main_path}/facebook-clean/user_id2urls_expanded_domains.parquet")
    df_facebook = pl.read_parquet(data_path).with_columns([
        pl.col('user_id').cast(pl.Utf8)  # Cast user_id to string
    ])
    if verbose: display(df_facebook.head(2))


    # Filtering
    COL = "urls_expanded"
    
    df_twitter = df_twitter.filter(
        pl.col(COL).list.len() > MIN_URLS_PER_USER
    )
    
    df_telegram = df_telegram.filter(
        pl.col(COL).list.len() > MIN_URLS_PER_USER
    )
        
    df_facebook = df_facebook.filter(
        pl.col(COL).list.len() > MIN_URLS_PER_USER
    )
    
    
    print(f"Twitter: {len(df_twitter):,}, Telegram: {len(df_telegram):,}, Facebook: {len(df_facebook):,}")


    # Combine
    # Remove unrelated users for Twitter
    if UNRELATED_NODES['twitter']:
        df_twitter = df_twitter.filter(~pl.col("user_id").is_in(UNRELATED_NODES['twitter']))
    # Add the user suffix for Twitter
    df_twitter_modified = df_twitter.with_columns(
        (pl.col("user_id").cast(pl.Utf8) + "_twitter_user").alias("user_id")
    ).select(["user_id", "profile_url", COL, "domains_expanded"])
    
    # Remove unrelated users for Telegram
    if UNRELATED_NODES['telegram']:
        df_telegram = df_telegram.filter(~pl.col("channel_id").is_in(UNRELATED_NODES['telegram']))
    # Add the user suffix for Telegram
    df_telegram_modified = df_telegram.with_columns(
        (pl.col("channel_id").cast(pl.Utf8) + "_telegram_channel").alias("user_id")
    ).select(["user_id", "profile_url", COL, "domains_expanded"])

    
    # Remove unrelated users for Facebook
    if UNRELATED_NODES['facebook']:
        df_facebook = df_facebook.filter(~pl.col("user_id").is_in(UNRELATED_NODES['facebook']))
    # Add the user suffix for Facebook
    df_facebook_modified = df_facebook.with_columns(
        (pl.col("user_id").cast(pl.Utf8) + "_facebook_user").alias("user_id")
    ).select(["user_id", "profile_url", COL, "domains_expanded"])
    
    
    # Concatenate both dataframes
    df_combined = pl.concat([df_twitter_modified, df_telegram_modified, df_facebook_modified])

    return df_combined


def get_user_df(platform):
    if platform == 'telegram':
        data_path = Path(f"{main_path}/telegram-clean/channel2urls_expanded_domains.parquet")
        df = pl.read_parquet(data_path).with_columns([
            pl.col("channel_id").cast(pl.Utf8)
        ]).select(["channel_id", "profile_url", "urls_expanded", "domains_expanded"])
    else:
        data_path = Path(ff"{main_path}/{platform}-clean/user_id2urls_expanded_domains.parquet")
        df = pl.read_parquet(data_path).with_columns([
            pl.col("user_id").cast(pl.Utf8)
        ]).select(["user_id", "profile_url", "urls_expanded", "domains_expanded"])
    return df


def get_messages_df(platform):
    if platform == 'telegram':
        data_path = Path(f"{main_path}/telegram-clean/channel2message_cleaned.parquet")
        df = pl.read_parquet(data_path, columns=["channel_id", 'profile_url', 'message', 'views', 'forwards', 'replies']).with_columns([
            pl.col("channel_id").cast(pl.Utf8),
            (pl.col('views') + pl.col('forwards') + pl.col('replies')).alias('sum_interactions').cast(pl.Int64)
        ]).select(["channel_id", 'profile_url', 'sum_interactions', 'message'])
    else:
        if platform == 'twitter':
            data_path = Path(ff"{main_path}/{platform}-clean/user_id2message_cleaned.parquet")
            df = pl.read_parquet(data_path, columns=["user_id", 'profile_url', 'message', 'likeCount', 'retweetCount', 'quoteCount', 'replyCount']).with_columns([
                pl.col(USER_LABEL).cast(pl.Utf8),
                (pl.col('likeCount') + pl.col('retweetCount') + pl.col('quoteCount') + pl.col('replyCount')).alias('sum_interactions').cast(pl.Int64)
            ]).select(["user_id", 'profile_url', 'sum_interactions', 'message'])
        elif platform=='facebook':
            data_path = Path(ff"{main_path}/{platform}-clean/{platform}_cleaned.parquet")
            df = pl.read_parquet(data_path, columns=["user_id", 'profile_url', 'message',  'actual_like', 'actual_share', 'actual_comment']).with_columns([
                pl.col(USER_LABEL).cast(pl.Utf8),
                (pl.col('actual_like') + pl.col('actual_share') + pl.col('actual_comment')).alias('sum_interactions').cast(pl.Int64)
            ]).select(["user_id", 'profile_url', 'sum_interactions', 'message'])
        else:
            raise Exception(platform)
    
    return df

def get_network(platform):
    TYPE_OF_NETWORK = 'url'
    hashed_params = "-".join([f"{k}_{v}" for k, v in params[TYPE_OF_NETWORK].items()])
    output_name = f"{platform}-coordinated-params-{hashed_params}"
    filename = f"./cross-platform/grid-search/{output_name}/{output_name}-networkx-graph.gexf"
    return nx.read_gexf(filename)

In [44]:
from pathlib import Path
import polars as pl
import pandas as pd
import networkx as nx
COMMON_LINKS = {"x.com", "t.me", "youtu.be", "www.youtube.com", "youtube.com", "www.tiktok.com", 
                "twitter.com", "t.co", "www.instagram.com", "vm.tiktok.com", "open.substack.com", 
                "vm.tiktok.com", "www.facebook.com", "www.msn.com", "m.youtube.com",
                "www.google.com", "goo.gl", "maps.google.com","rumble.com",
                "m.facebook.com", "l.facebook.com", "venmo.com", 
                "www.paypal.com", "www.reddit.com", "old.reddit.com",
                "www.newsbreakapp.com", "go.shr.lc"}


platform2nodes = {}
platform2domains = {}

for platform, params in platform2parameters.items():
    print("\n", "="*25)
    print(f"\n\n ------> {platform}")
    USER_LABEL = "channel_id" if platform == "telegram" else "user_id"
    G_filtered = get_network(platform)
    nodes = set(G_filtered.nodes())
    
    if platform == 'cross':
        all_iodf = []
        all_iodf_messages = []
        mapping = {x: x.split("_")[-3] for x in nodes}
        G_filtered = nx.relabel_nodes(G_filtered, mapping)
        nodes = set(G_filtered.nodes())

        print("\n\n--- CROSS NODES: ")
        print(nodes)
        print("---\n\n")

        for platform2 in ['telegram', 'twitter', 'facebook']:
            USER_LABEL2 = "channel_id" if platform2 == "telegram" else "user_id"
            iodf = get_user_df(platform2)
            iodf = iodf.filter(pl.col(USER_LABEL2).is_in(nodes)).rename({USER_LABEL2: "user_id"})
            all_iodf.append(iodf)
            if iodf.is_empty():
                print(f"Cross analysis did not find any user in {platform2}")
            
            iodf_messages = get_messages_df(platform2)
            iodf_messages = iodf_messages.filter(pl.col(USER_LABEL2).is_in(nodes)).rename({USER_LABEL2: "user_id"})
            all_iodf_messages.append(iodf_messages)
            
        iodf = pl.concat(all_iodf)
        iodf_messages = pl.concat(all_iodf_messages)
    else:
        iodf = get_user_df(platform)
        iodf = iodf.filter(pl.col(USER_LABEL).is_in(nodes))
        iodf_messages = get_messages_df(platform)
        iodf_messages = iodf_messages.filter(pl.col(USER_LABEL).is_in(nodes))

    platform2nodes[platform] = nodes
    
    # ----> DOMAINS
    print("--> DOMAINS")
    for i, users_component in enumerate(nx.connected_components(G_filtered)):
        print(f"Component: {i}")
        print(f"{len(users_component)} Users: {users_component}")
        urls_df = count_unique_terms(iodf.to_pandas()[iodf.to_pandas()[USER_LABEL].isin(users_component)], 'domains_expanded')
        merged_urls_df = merge_values_by_case_insensitivity(urls_df)
        filtered_df = merged_urls_df[~merged_urls_df.value.isin(COMMON_LINKS)].nlargest(5, 'count')
        display(filtered_df)
        
        # Add the 'Credibility' column
        filtered_df['Credibility'] = "NA"
        filtered_df['Leaning'] = "NA"
        
        # Modify the 'Reachable URL' column to include the clickable LaTeX hyperlinks
        filtered_df['Reachable URL'] = filtered_df['value'].apply(lambda x: f'\\href{{{x}}}{{{x}}}')
        
        # Generate the LaTeX table with the appropriate columns
        latex_table = filtered_df[['Reachable URL', 'count', 'Credibility', 'Leaning']].to_latex(index=False, column_format='c|c|c|c', 
                                           header=['Domain', 'Shares', 'Credibility', 'Leaning'], 
                                       escape=False)
        # Collect domains
        platform2domains[(platform, i)] = set(merged_urls_df.value.unique())
        
        # Print table
        print(latex_table)

        
        # ---> RUSSIA DOMAINS
        count = 0
        set_of_domains = set()
        set_of_domains_full_url = set()
        for russ_dom in russia_media:
            for dom in merged_urls_df.value.unique():
                if "."+russ_dom+"." in dom:
                    count += 1
                    set_of_domains.add(russ_dom)
                    set_of_domains_full_url.add(dom)
                    #print("Domain:", russ_dom, dom)
        print(f"\n{count} russian domains  / {merged_urls_df.value.nunique()}")
        print(set_of_domains)
        display(merged_urls_df[merged_urls_df.value.isin(set_of_domains_full_url)])
       

    # ----> URLS
    print("--> URLS")
    for i, users_component in enumerate(nx.connected_components(G_filtered)):
        print(f"Component: {i}")
        urls_df = count_unique_terms(iodf.to_pandas()[iodf.to_pandas()[USER_LABEL].isin(users_component)], 'urls_expanded')
        merged_urls_df = merge_values_by_case_insensitivity(urls_df)
        merged_urls_df = merged_urls_df[~merged_urls_df.value.str.contains('t.me/')]
        display(merged_urls_df.nlargest(5, 'count'))
        print(merged_urls_df.nlargest(5, 'count').value.to_list())




    
    # [USERS]:
    print("\n--> USERS")
    for i, users_component in enumerate(nx.connected_components(G_filtered)):
        print("\n Component", i)
        degree_dict = dict(G_filtered.degree(users_component))
        sorted_nodes_by_degree = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)
        top_nodes = {node_id for node_id, degree in sorted_nodes_by_degree[:5]}
        print(iodf.filter(pl.col(USER_LABEL).is_in(top_nodes)).select('profile_url').to_series().to_list())

    # [MESSAGES]
    print("\n--> MESSAGES")
    for i, users_component in enumerate(nx.connected_components(G_filtered)):
        print("\n Component", i)
        # Filter messages, remove duplicates, and get top 10 sorted by 'sum_interactions'
        messages = (
            iodf_messages
            .filter(pl.col(USER_LABEL).is_in(users_component))  # Filter by users
            .sort('sum_interactions', descending=True)          # Sort by sum_interactions
            .select('message')                                  # Select the message column
            .unique()                                           # Remove duplicates
            .head(10)                                           # Get top 10 messages
            .to_series()
            .to_list()
        )
        
        # Print the unique messages
        for j, mes in enumerate(messages):
            print(j, "----", mes)

# Bridge twitter: https://x.com/intent/user?user_id=1361744068066353156
# Bridges telegram: 1713271586, 1257157877, 1362482073, 1481307045 




 ------> cross


--- CROSS NODES: 
{'59643287', '1641929188369395713', '940430173517811714', '1665107911243071488', '1244107710', '1413401799', '1743361744', '19211550', '1568299437', '1586802839128522753', '1598823026707124227', '1470757978', '1713271586', '1378626018', '1300686753', 'i0v0x66w', '1312239941', '1672235756', '1783482370', '1443589832', '1310767622351323137', '1391253363', '849342841', '1506646784', '1556411797', '1675644680678125570', '1636472491416883200', '1586104597713752064', '1520230767', '18856867', '11134252', '249666995', '1374926040', '250454150', '1526112366', '1314675598850220039', '1602707192867860480', '996571436474163200', '1231902969', '1493032828195377153', 'o0zb2jhal', '1952165884', '1405404884', '390127278', '24841259', '1417164314', '1402888043', '1366719600', '1211378130', '1072069110413037569', '1567338712740106240', '1780570221276774400', '1073005498557845506', '1598123408621015042', '1481307045', '1625437643', '1304156557', '1317678756', '16533

,value,count
10075,www.thegatewaypundit.com,23513
10883,www.zerohedge.com,8331
5364,truthsocial.com,7796
3659,nypost.com,6779
10069,www.theepochtimes.com,6241


\begin{tabular}{c|c|c|c}
\toprule
Domain & Shares & Credibility & Leaning \\
\midrule
\href{www.thegatewaypundit.com}{www.thegatewaypundit.com} & 23513 & NA & NA \\
\href{www.zerohedge.com}{www.zerohedge.com} & 8331 & NA & NA \\
\href{truthsocial.com}{truthsocial.com} & 7796 & NA & NA \\
\href{nypost.com}{nypost.com} & 6779 & NA & NA \\
\href{www.theepochtimes.com}{www.theepochtimes.com} & 6241 & NA & NA \\
\bottomrule
\end{tabular}


7 russian domains  / 10979
{'gazeta', 'mil', 'tv5', 'rbc', 'ruptly'}


,value,count
3503,news.tv5.com.ph,1
3556,newsukraine.rbc.ua,8
7516,www.gazeta.ru,7
9028,www.nzdf.mil.nz,1
9400,www.rbc.ru,2
9541,www.ruptly.tv,2117
9542,www.ruptly.video,2


--> URLS
Component: 0


...

\begin{tabular}{c|c|c|c}
\toprule
Domain & Shares & Credibility & Leaning \\
\midrule
\href{gorightnews.com}{gorightnews.com} & 15 & NA & NA \\
\href{cash.app}{cash.app} & 10 & NA & NA \\
\href{kick.com}{kick.com} & 10 & NA & NA \\
\href{peterboykin.com}{peterboykin.com} & 10 & NA & NA \\
\href{archive.is}{archive.is} & 3 & NA & NA \\
\bottomrule
\end{tabular}


0 russian domains  / 20
set()


,value,count


--> URLS
Component: 0


,value,count
0,http://cash.app/$peterboykin1,10
1,http://kick.com/peterboykin,10
2,http://rumble.com/gorightnews,10
7,https://gorightnews.com/donations/support-gori...,10
10,https://peterboykin.com,10


['http://cash.app/$peterboykin1', 'http://kick.com/peterboykin', 'http://rumble.com/gorightnews', 'https://gorightnews.com/donations/support-gorightnews/', 'https://peterboykin.com']

--> USERS

 Component 0
['https://www.facebook.com/GayRightNews/', 'https://www.facebook.com/Gays4Trump/', 'https://www.facebook.com/GoRightForAmerica/', 'https://www.facebook.com/TheQiewCom/', 'https://www.facebook.com/WalkAwayUSA/']

--> MESSAGES

 Component 0
0 ---- Joe Biden White House Spins 'Cheap Fakes' to Deflect Attention from Biden's Gaffes https://gorightnews.com/joe-biden-white-house-spins-cheap-fakes-to-deflect-attention-from-bidens-gaffes/ White House creates term 'Cheap Fakes' to confuse obscure Biden gaffes #GoRightNews In a bold attempt to manage the public perception of President Joe Biden's cognitive abilities, the White House has coined a new term: "cheap fakes." This term appears designed to dismiss viral clips highlighting Biden's frequent gaffes as bad faith attacks rather than genu